Ячейка с импортами и задание общих переменных запускаются в обоих сценариях.

Ячейка с версией 3 -- версия с одноядерной обработкой.

Ячейка с версией 4 и следующая за ней -- мультипроцессинг (не работает)

In [1]:
import pysam
import array
import argparse
import numpy as np
import pandas as pd
from concurrent.futures import ProcessPoolExecutor

In [ ]:
# parser = argparse.ArgumentParser()
# parser.add_argument("input_bam", type=str)
# parser.add_argument("output_bam", type=str)
# parser.add_argument("snp_file", type=str)
# arguments = parser.parse_args()
# input_bam = arguments.input_bam
# output_bam = arguments.output_bam
# snp_file = arguments.snp_file

In [2]:
input_bam = "../../interpret/genome_38.bam"
output_bam = "../../interpret/genome_38_mod"
snp_file = "../../interpret/snps.csv"
n_proc = 4

In [14]:
# Версия кода 3 (много ридов, много хромосом, функция)
snips = pd.read_csv(snp_file)

with pysam.AlignmentFile(input_bam, "rc") as samfile_input, pysam.AlignmentFile(output_bam, "wc", template=samfile_input) as samfile_output:
    
    # Инициализируем список хромосом
    chroms = samfile_input.references
    
    for chrom in chroms:
        # Если в хромосоме не планируются SNP, не трогаем ее
        if chrom not in set(snips["chromosome"]):
            reads = samfile_input.fetch(chrom)
            for read in reads:
                samfile_output.write(read)

        # А если планируются, трогаем
        else:

            # Отбираем SNP для данной хромосомы
            snp_subset = snips.query("chromosome==@chrom")
            chrom_snps = snp_subset["position"]
            first_p, last_p = chrom_snps.min(), chrom_snps.max()

            # Риды до SNP-позиций
            reads = samfile_input.fetch(chrom, 0, first_p)
            for read in reads:

                # Страховка для ситуаций, где картировался только один рид из пары
                if read.reference_end is None:
                    if (read.reference_start + read.query_length - 1) < first_p:
                        samfile_output.write(read)
                else:
                    if read.reference_end < first_p:
                        samfile_output.write(read)             

            # Риды, содержащие SNP-позиции
            reads = samfile_input.fetch(chrom, first_p, last_p + 1)
            for read in reads:
                number_list = read.get_reference_positions()

                # Сохраняем в переменные данные исходных ридов
                rs = read.query_sequence
                rq = read.query_qualities
                rc = read.cigarstring

                for number in range(len(chrom_snps)):
                    if chrom_snps.iloc[number] in number_list:

                        # Вычисляем индекс интересующего нуклеотида
                        ind = number_list.index(chrom_snps.iloc[number])

                        # Создаем данные модифицированных ридов
                        rs = rs[:ind] + snp_subset["nucleotide"].iloc[number] + rs[ind + 1:]
                        rq[ind] = snp_subset["quality"].iloc[number]

                # Изменяем парметры ридов, вносим в новый файл
                read.query_sequence = rs
                read.query_qualities = rq           
                samfile_output.write(read)

            # Риды после SNP-позиций
            reads = samfile_input.fetch(chrom, last_p + 1)
            for read in reads:
                if read.reference_start > last_p:
                    samfile_output.write(read)

        # Записываем невыровнявшиеся риды в конец
        reads = samfile_input.fetch("*")
        for read in reads:
            samfile_output.write(read)


[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_uncompress_block] Block CRC32 failure
[E::cram_next_slice] Failure to decode slice
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_unc

[E::cram_read_container] Container header CRC32 failure
[E::cram_uncompress_block] Block CRC32 failure
[E::cram_next_slice] Failure to decode slice
[E::cram_read_container] Container header CRC32 failure
[E::cram_uncompress_block] Block CRC32 failure
[E::cram_next_slice] Failure to decode slice
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_next_slice] Failure to decode slice
[E::cram_next_slice] Failure to decode slice
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_uncompress_block] Block CRC32 failure
[E::cram_next_slice] Failure to decode slice
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] [E::cram_read_container] Container header CRC32 failureContainer header CRC32 failure

[E::cram_uncompres

[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_next_slice] Failure to decode slice
[E::cram_next_slice] Failure to decode slice
[E::cram_next_slice] Failure to decode slice
[E::cram_next_slice] Failure to decode slice
[E::cram_read_container] Container header CRC32 failure
[E::cram_next_slice] Failure to decode slice
[E::cram_next_slice] Failure to decode slice
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_next_slice] Failure to decode slice
[E::cram_next_slice] Failure to decode slice
[E::cram_uncompress_block] Block CRC32 failure
[E::cram_next_slice] Failure to decode slice
[E::cram_read_container] Container header CRC32 failure
[E::cram_next_slice] Failure to decode slice
[E::cram_next_slice] Failure to decode slice
[E::cram_read_container] Container header CRC32 failure
[E::cram_n

[E::cram_next_slice] Failure to decode slice
[E::cram_next_slice] Failure to decode slice
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_uncompress_block] Block CRC32 failure
[E::cram_next_slice] Failure to decode slice
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_uncompress_block] Block CRC32 failure
[E::cram_next_slice] Failure to decode slice
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failur

[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_uncompress_block] Block CRC32 failure
[E::cram_next_slice] Failure to decode slice
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] [E::cram_read_container] Container header CRC32 failureContainer header CRC32 failure

[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_rea

[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_next_slice] Failure to decode slice
[E::cram_uncompress_block] Block CRC32 failure
[E::cram_next_slice] Failure to decode slice
[E::cram_next_slice] Failure to decode slice
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_next_slice] Failure to decode slice
[E::cram_next_slice] Failure to decode slice
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure


[E::cram_read_container] Container header CRC32 failure
[E::cram_next_slice] Failure to decode slice
[E::cram_next_slice] Failure to decode slice
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_uncompress_block] Block CRC32 failure
[E::cram_next_slice] Failure to decode slice
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_read_container] Container header CRC32 failure
[E::cram_uncompress_block] Block CRC32 failure
[E::cram_next_slice] Failure to decode slic

In [3]:
# Версия 4 Компиляция БАМов + мультипроцессинг
def radioactivnye_ludi(chrom):
    # print(f"хрома: {chrom}, {snips['chromosome']}")
    with pysam.AlignmentFile(f"{output_bam}_{chrom}.bam", "wb", template=samfile_input) as samfile_output:
        # Если в хромосоме не планируются SNP, не трогаем ее
        if chrom not in set(snips["chromosome"]):
            reads = samfile_input.fetch(chrom)
            for read in reads:
                # print(f"{chrom}: {read}")
                samfile_output.write(read)

        # А если планируются, трогаем
        else:

            # Отбираем SNP для данной хромосомы
            snp_subset = snips.query("chromosome==@chrom")
            chrom_snps = snp_subset["position"]
            first_p, last_p = chrom_snps.min(), chrom_snps.max()

            # Риды до SNP-позиций
            reads = samfile_input.fetch(chrom, 0, first_p)
            for read in reads:

                # Страховка для ситуаций, где картировался только один рид из пары
                if read.reference_end is None:
                    if (read.reference_start + read.query_length - 1) < first_p:
                        samfile_output.write(read)
                else:
                    if read.reference_end < first_p:
                        samfile_output.write(read)             

            # Риды, содержащие SNP-позиции
            reads = samfile_input.fetch(chrom, first_p, last_p + 1)
            for read in reads:
                number_list = read.get_reference_positions()

                # Сохраняем в переменные данные исходных ридов
                rs = read.query_sequence
                rq = read.query_qualities
                rc = read.cigarstring

                for number in range(len(chrom_snps)):
                    if chrom_snps.iloc[number] in number_list:

                        # Вычисляем индекс интересующего нуклеотида
                        ind = number_list.index(chrom_snps.iloc[number])

                        # Создаем данные модифицированных ридов
                        rs = rs[:ind] + snp_subset["nucleotide"].iloc[number] + rs[ind + 1:]
                        rq[ind] = snp_subset["quality"].iloc[number]

                # Изменяем парметры ридов, вносим в новый файл
                read.query_sequence = rs
                read.query_qualities = rq           
                samfile_output.write(read)

            # Риды после SNP-позиций
            reads = samfile_input.fetch(chrom, last_p + 1)
            for read in reads:
                if read.reference_start > last_p:
                    samfile_output.write(read)

        # Записываем невыровнявшиеся риды в конец
        reads = samfile_input.fetch("*")
        for read in reads:
            samfile_output.write(read)

In [4]:
snips = pd.read_csv(snp_file)

with pysam.AlignmentFile(input_bam, "rb") as samfile_input:
    
    # Инициализируем список хромосом
    chroms = samfile_input.references
    
    #for chrom in chroms:
        #radioactivnye_ludi(chrom)
        
    with ProcessPoolExecutor(n_proc) as pool:                               
        pool.map(radioactivnye_ludi, chroms)

[E::bgzf_read_block] Invalid BGZF header at offset 75006839891
[E::bgzf_read] Read block operation failed with error 2 after 0 of 4 bytes
[E::bgzf_uncompress] Inflate operation failed: invalid distance too far back
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read_block] Invalid BGZF header at offset 75010508837
[E::bgzf_read] Read block operation failed with error 2 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with

[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block

[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block

[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block

[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block

...

[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block

[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block

[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block

[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block

[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 3 after 0 of 4 bytes
[E::bgzf_read] Read block operation failed with error 1 after 0 of 4 bytes
[E::bgzf_read] Read block